In [1]:
# 3) Buzz factor: currency Name frequency in search engine(?) results, sort into News?, blogs?, ect?
# --------NEED DATES
# --------Get enough text to maybe do some sentiment analysis with NLP?

In [11]:
import numpy as np
import pandas as pd
import requests as re
import time

In [45]:
def create_epoch(date, clock):
    """Given date (format = ‘mm/dd/yyyy’) and clock (format = 'HH:MM:SS') return the Epoch."""
    date_time = f"{date} {clock}"
    pattern = '%m/%d/%Y %H:%M:%S'
    return int(time.mktime(time.strptime(date_time, pattern)))

In [46]:
def fetch_reddit_data(start, end, coin, size = 50, fields = [
    'num_comments', 'title', 'score', 'upvote_ratio', 'subreddit']):
    """
    Given a date range and coin, get size (max 500) reddit posts
    about that coin sorted from highest to lowest score
    """
    
    base_url = 'https://api.pushshift.io/reddit/search/submission/'
    params = {
        'title': coin,
        'fields': fields,
        'sort_type': 'score',
        'after':  create_epoch(start, '00:00:01'),
        'before': create_epoch(end, '23:59:59'),
        'size': size
    }
    return re.get(base_url, params).json()['data']

In [51]:
fetch_reddit_data('9/20/2020', '9/28/2020', 'Bitcoin')

[{'num_comments': 2,
  'score': 0,
  'subreddit': 'BitcoinBeginners',
  'title': 'Really Good place for Crypto Beginners to Learn about Bitcoin and Cryptos',
  'upvote_ratio': 0.5},
 {'num_comments': 5,
  'score': 0,
  'subreddit': 'Bitcoin',
  'title': '1 billion in bitcoin on Facebook',
  'upvote_ratio': 0.5},
 {'num_comments': 4,
  'score': 0,
  'subreddit': 'sportsbook',
  'title': 'Bitcoin Cashout On MyBookie',
  'upvote_ratio': 0.5},
 {'num_comments': 6,
  'score': 0,
  'subreddit': 'NoStupidQuestions',
  'title': 'Is there a good sub for beginner questions about trading things like Stocks, Forex, Bitcoin etc.?',
  'upvote_ratio': 0.5},
 {'num_comments': 0,
  'score': 1,
  'subreddit': 'CertificadoDigital',
  'title': 'Bitcoin fracassou? Qual é o futuro do Bitcoin? - Dicas sobre Bitcoin - Ganhe dinheiro',
  'upvote_ratio': 1.0},
 {'num_comments': 0,
  'score': 1,
  'subreddit': 'u_Afanasy228',
  'title': 'Best Bitcoin Generator 2020 No Fee Get 0.016 BTC Per Day',
  'upvote_ratio'